In [1]:
# System modules
import os
import sys
from dotenv import load_dotenv
from time import time
import json

import requests
import pandas as pd

import logging
import requests
from hashlib import md5
import json

import pandas as pd


import plotly.graph_objects as go
import plotly.express as px

import nltk

# Append source directory to system path
src_path = os.path.abspath(os.path.join("../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

# Helper functions
import data.helpers as data_helpers


load_dotenv()  # take environment variables from .env.

YELP_CLIENT_ID = os.getenv("YELP_CLIENT_ID")
YELP_API_KEY = os.getenv("YELP_API_KEY")


In [2]:
def yelp_business_search_by_location(
    location: str = "Paris", count: int = 200
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Get Yelp data from API.

    Params:
        location: str
        count: int

    Returns:
        json
    """
    headers = {
        "Authorization": f"Bearer {YELP_API_KEY}",
    }
    limit = 50

    businesses = pd.DataFrame()
    reviews = pd.DataFrame()
    photos = pd.DataFrame()
    for offset in range(0, count, limit):
        businesses_request = requests.get(
            "https://api.yelp.com/v3/businesses/search",
            headers=headers,
            params={
                "location": location,
                "limit": limit,
                "offset": offset,
            },
        )

        if businesses_request.status_code == 200:
            businesses_data = businesses_request.json()
            businesses = businesses.append(
                pd.DataFrame(businesses_data["businesses"]), ignore_index=True
            )

            for business in businesses_data["businesses"]:
                business_detail_request = requests.get(
                    f"https://api.yelp.com/v3/businesses/{business['id']}",
                    headers=headers,
                )
                if business_detail_request.status_code == 200:
                    business_detail_data = business_detail_request.json()
                    photos = photos.append(
                        pd.DataFrame(business_detail_data["photos"]),
                        ignore_index=True,
                    )
                else:
                    raise Exception(
                        f"Yelp API request failed with status code \
                             { business_detail_request.status_code }. \
                                 Response text: { business_detail_request.text }"
                    )

            for business in businesses_data["businesses"]:
                business_reviews_request = requests.get(
                    f"https://api.yelp.com/v3/businesses/{business['id']}/reviews",
                    headers=headers,
                )
                if business_reviews_request.status_code == 200:
                    business_reviews_data = business_reviews_request.json()
                    reviews = reviews.append(
                        pd.DataFrame(business_reviews_data["reviews"]),
                        ignore_index=True,
                    )
                else:
                    raise Exception(
                        f"Yelp API request failed with status code { business_reviews_request.status_code }. Response text: { business_reviews_request.text }"
                    )

        else:
            raise Exception(
                f"Yelp API request failed with status code { businesses_request.status_code }. Response text: { businesses_request.text }"
            )

    return businesses, reviews, photos


In [3]:
def get_yelp_data(
    locations: list[str] = [
        "Paris",
        "New York City",
        "Tokyo",
        "Rio de Janeiro",
        "Sydney",
    ],
    category: str = "restaurants",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Get Yelp data from API.

    - iterate over the locations
        - build a GraphQL query to get the data
        - send the query to the Yelp API
        - parse the response
        - append to the dataframe
    - return the dataframes

    Params:
        locations: str[] (default: ["Paris"]) - List of Yelp locations to search
        category: str (default: "restaurants") - Yelp category (see https://www.yelp.com/developers/documentation/v3/all_category_list)

    Returns:
        businesses: pd.DataFrame - businesses data from Yelp API request
        reviews: pd.DataFrame - reviews data from Yelp API request
        photos: pd.DataFrame - photos data from Yelp API request
    """
    # businesses data (see https://www.yelp.com/developers/graphql/objects/business)
    businesses = pd.DataFrame(
        columns=[
            "business_alias",  # Unique Yelp alias of this business.
            "business_review_count",  # Total number of reviews for this business.
            "business_rating",  # Rating of the business, which is an average of the ratings of all reviews.
            "business_price",  # Price range of the business, from "$" to "$$$$" (inclusive).
            "business_city",  # City of this business.
            "business_state",  # ISO 3166-2 (with a few exceptions) state code of this business (see https://www.yelp.com/developers/documentation/v3/state_codes).
            "business_postal_code",  # Postal code of this business (see https://en.wikipedia.org/wiki/Postal_code)
            "business_country",  # ISO 3166-1 alpha-2 country code of this business.
            "business_latitude",  # Latitude of the business.
            "business_longitude",  # Longitude of the business.
            "business_categories",  # List of categories the business belongs to.
            "business_parent_categories",  # List of parent categories the business belongs to.
        ]
    )
    reviews = pd.DataFrame(
        columns=[
            "business_alias",  # Unique Yelp alias of the business.
            "review_text",  # Text excerpt of this review.
            "review_rating",  # Rating of this review.
        ]
    )
    photos = pd.DataFrame(
        columns=[
            "business_alias",  # Unique Yelp alias of the business.
            "photo_url",  # URL of the photo.
        ]
    )

    # Yelp's GraphQL endpoint
    url = "https://api.yelp.com/v3/graphql"
    # Request headers
    headers = {
        "Authorization": f"Bearer {YELP_API_KEY}",
        "Content-Type": "application/graphql",
    }
    count = 200  # Yelp's GraphQL API returns a maximum of 240 total results
    limit = 50  # Yelp's GraphQL API returns a maximum of 50 results per request

    for location in locations:
        for offset in range(0, count, limit):
            # Build the GraphQL query
            query = f'{{\n\
        search(categories: "{ category }", location: "{ location }", offset: { offset }, limit:  { limit }) {{\n\
            business {{\n\
                alias\n\
                review_count\n\
                rating\n\
                price\n\
                location {{\n\
                    city\n\
                    state\n\
                    postal_code\n\
                    country\n\
                }}\n\
                coordinates {{\n\
                    latitude\n\
                    longitude\n\
                }}\n\
                categories {{\n\
                    alias\n\
                    parent_categories {{\n\
                        alias\n\
                    }}\n\
                }}\n\
                photos\n\
                reviews {{\n\
                    text\n\
                    rating\n\
                }}\n\
            }}\n\
        }}\n\
    }}'
            # Send the query to the Yelp API
            response = requests.post(url, headers=headers, data=query)
            # Parse the response
            if not response.status_code == 200:
                raise Exception(
                    f"Yelp API request failed with status code { response.status_code }. Response text: { response.text }"
                )

            # Parse the response
            data = response.json()

            if "errors" in data:
                raise Exception(
                    f"Yelp API request failed with errors: { data['errors'] }"
                )

            for business in (
                data.get("data", {}).get("search", {}).get("business", [])
            ):
                # Add the business data to the dataframe
                businesses = businesses.append(
                    {
                        "business_alias": business.get("alias"),
                        "business_review_count": business.get("review_count"),
                        "business_rating": business.get("rating"),
                        "business_price": len(  # count the number of characters ($, €, ...)
                            business.get("price")
                        )
                        if business.get("price") is not None
                        else 0,
                        "business_city": business.get("location", {}).get(
                            "city"
                        ),
                        "business_state": business.get("location", {}).get(
                            "state"
                        ),
                        "business_postal_code": business.get(
                            "location", {}
                        ).get("postal_code"),
                        "business_country": business.get("location", {}).get(
                            "country"
                        ),
                        "business_latitude": business.get(
                            "coordinates", {}
                        ).get("latitude"),
                        "business_longitude": business.get(
                            "coordinates", {}
                        ).get("longitude"),
                        "business_categories": json.dumps(
                            list(
                                set(  # keep unique values
                                    [
                                        cat.get("alias")
                                        for cat in business.get(
                                            "categories", []
                                        )
                                    ]
                                )
                            )
                        ),
                        "business_parent_categories": json.dumps(
                            list(
                                set(  # keep unique values
                                    [
                                        parent_cat.get("alias")
                                        for cat in business.get(
                                            "categories", []
                                        )
                                        for parent_cat in cat.get(
                                            "parent_categories", []
                                        )
                                    ]
                                )
                            )
                        ),
                    },
                    ignore_index=True,
                )

                for photo in business.get("photos", []) or []:
                    # Add the photo data to the dataframe
                    photos = photos.append(
                        {
                            "business_alias": business.get("alias"),
                            "photo_url": photo,
                        },
                        ignore_index=True,
                    )

                for review in business.get("reviews", []) or []:
                    # Add the review data to the dataframe
                    reviews = reviews.append(
                        {
                            "business_alias": business.get("alias"),
                            "review_text": review.get("text"),
                            "review_rating": review.get("rating"),
                        },
                        ignore_index=True,
                    )

    # Return the dataframes
    return businesses, reviews, photos


def download_photos(
    photos: pd.DataFrame,
    target_path: str,
) -> None:
    # Check if content path exists
    if not os.path.exists(target_path):
        logging.info(f"Creating {target_path}")
        os.makedirs(target_path)

    for photo in photos.itertuples(index=False):
        file_name = f"{ photo.business_alias }_{ md5(photo.photo_url.encode('utf-8')).hexdigest() }.jpg"
        file_path = os.path.join(target_path, file_name)

        if not os.path.exists(file_path):
            request = requests.get(photo.photo_url)
            if not 200 == request.status_code:
                logging.warning(
                    f"Photo URL : { photo.photo_url }\nYelp API request failed with status code: { request.status_code }.\nResponse text: { request.text }"
                )
                continue

            photo_data = request.content
            with open(file_path, "wb") as f:
                f.write(photo_data)


In [4]:
data_path = "../data/raw/"
businesses_csv_path = os.path.join(data_path, "businesses.csv")
reviews_csv_path = os.path.join(data_path, "reviews.csv")
photos_csv_path = os.path.join(data_path, "photos.csv")

if not os.path.exists(data_path):
    logging.info(f"Creating {data_path}")
    os.makedirs(data_path)

if (
    os.path.exists(businesses_csv_path)
    and os.path.exists(reviews_csv_path)
    and os.path.exists(photos_csv_path)
):
    logging.info(f"Data already downloaded")
    businesses_df = pd.read_csv(businesses_csv_path)
    reviews_df = pd.read_csv(reviews_csv_path)
    photos_df = pd.read_csv(photos_csv_path)
else:
    logging.info("Saving data")
    businesses_df, reviews_df, photos_df = get_yelp_data()

    businesses_df.to_csv(businesses_csv_path, index=False)
    reviews_df.to_csv(reviews_csv_path, index=False)
    photos_df.to_csv(photos_csv_path, index=False)


# Fix dtypes
businesses_df["business_alias"] = businesses_df["business_alias"].astype(str)
businesses_df["business_review_count"] = businesses_df[
    "business_review_count"
].astype(int)
businesses_df["business_rating"] = businesses_df["business_rating"].astype(
    float
)
businesses_df["business_price"] = businesses_df["business_price"].astype(int)
businesses_df["business_city"] = businesses_df["business_city"].astype(str)
businesses_df["business_state"] = businesses_df["business_state"].astype(str)
businesses_df["business_postal_code"] = businesses_df[
    "business_postal_code"
].astype(str)
businesses_df["business_country"] = businesses_df["business_country"].astype(
    str
)
businesses_df["business_latitude"] = businesses_df["business_latitude"].astype(
    float
)
businesses_df["business_longitude"] = businesses_df[
    "business_longitude"
].astype(float)
businesses_df["business_categories"] = businesses_df[
    "business_categories"
].astype(str)
businesses_df["business_parent_categories"] = businesses_df[
    "business_parent_categories"
].astype(str)

reviews_df["business_alias"] = reviews_df["business_alias"].astype(str)
reviews_df["review_text"] = reviews_df["review_text"].astype(str)
reviews_df["review_rating"] = reviews_df["review_rating"].astype(float)

photos_df["business_alias"] = photos_df["business_alias"].astype(str)
photos_df["photo_url"] = photos_df["photo_url"].astype(str)


# Reduce memory usage
businesses_df = data_helpers.reduce_dataframe_memory_usage(businesses_df)
reviews_df = data_helpers.reduce_dataframe_memory_usage(reviews_df)
photos_df = data_helpers.reduce_dataframe_memory_usage(photos_df)


In [5]:
businesses_df.head()


,business_alias,business_review_count,business_rating,business_price,business_city,business_state,business_postal_code,business_country,business_latitude,business_longitude,business_categories,business_parent_categories
0,le-comptoir-de-la-gastronomie-paris,1107,4.5,2,Paris,75,75001,FR,48.864517,2.345402,"[""french""]","[""restaurants""]"
1,l-as-du-fallafel-paris,1810,4.5,1,Paris,75,75004,FR,48.857498,2.359080,"[""kosher"", ""falafel"", ""sandwiches""]","[""restaurants"", ""mediterranean""]"
2,angelina-paris,1347,4.0,3,Paris,75,75001,FR,48.865093,2.328464,"[""tea"", ""cakeshop"", ""breakfast_brunch""]","[""restaurants"", ""food""]"
3,l-avant-comptoir-paris-3,612,4.5,2,Paris,75,75006,FR,48.852020,2.338800,"[""wine_bars"", ""tapas""]","[""bars"", ""restaurants""]"
4,la-coïncidence-paris-4,493,4.5,2,Paris,75,75116,FR,48.868107,2.284365,"[""french""]","[""restaurants""]"


In [6]:
businesses_df.describe(include="all")


,business_alias,business_review_count,business_rating,business_price,business_city,business_state,business_postal_code,business_country,business_latitude,business_longitude,business_categories,business_parent_categories
count,1000,1000.000000,1000.000000,1000.000000,1000,1000,1000,1000,1000.000000,1000.000000,1000,1000
unique,1000,NaN,NaN,NaN,40,6,301,5,NaN,NaN,530,50
top,le-comptoir-de-la-gastronomie-paris,NaN,NaN,NaN,Paris,13,2000,AU,NaN,NaN,"[""french""]","[""restaurants""]"
freq,1,NaN,NaN,NaN,200,200,139,200,NaN,NaN,56,551
mean,NaN,529.395000,4.270500,2.144000,NaN,NaN,NaN,NaN,13.688472,35.219700,NaN,NaN
std,NaN,1134.869631,0.353135,0.897814,NaN,NaN,NaN,NaN,34.823799,93.358452,NaN,NaN
min,NaN,6.000000,3.000000,0.000000,NaN,NaN,NaN,NaN,-33.897026,-74.016022,NaN,NaN
25%,NaN,30.000000,4.000000,2.000000,NaN,NaN,NaN,NaN,-22.983292,-43.218462,NaN,NaN
50%,NaN,71.000000,4.500000,2.000000,NaN,NaN,NaN,NaN,35.673141,2.340317,NaN,NaN
75%,NaN,265.750000,4.500000,3.000000,NaN,NaN,NaN,NaN,40.751259,139.770420,NaN,NaN


In [7]:
reviews_df.head()


,business_alias,review_text,review_rating
0,le-comptoir-de-la-gastronomie-paris,This review is from our 2019 trip. Shame on m...,5.0
1,le-comptoir-de-la-gastronomie-paris,This place def lives up the hype. Best French...,5.0
2,le-comptoir-de-la-gastronomie-paris,"While planning a friends trip to Paris, I came...",5.0
3,l-as-du-fallafel-paris,This is the best falafel sandwich I have ever ...,5.0
4,l-as-du-fallafel-paris,IMO this is a must try in Paris. Located in ...,5.0


In [8]:
reviews_df.describe(include="all")


,business_alias,review_text,review_rating
count,2928,2928,2928.000000
unique,990,2928,NaN
top,le-comptoir-de-la-gastronomie-paris,This review is from our 2019 trip. Shame on m...,NaN
freq,3,1,NaN
mean,NaN,NaN,4.396516
std,NaN,NaN,0.904879
min,NaN,NaN,1.000000
25%,NaN,NaN,4.000000
50%,NaN,NaN,5.000000
75%,NaN,NaN,5.000000


In [9]:
photos_df.head()


,business_alias,photo_url
0,le-comptoir-de-la-gastronomie-paris,https://s3-media2.fl.yelpcdn.com/bphoto/Je6THJ...
1,l-as-du-fallafel-paris,https://s3-media2.fl.yelpcdn.com/bphoto/wdIhzK...
2,angelina-paris,https://s3-media3.fl.yelpcdn.com/bphoto/DPM5TB...
3,l-avant-comptoir-paris-3,https://s3-media3.fl.yelpcdn.com/bphoto/mVwgxg...
4,la-coïncidence-paris-4,https://s3-media1.fl.yelpcdn.com/bphoto/QdrAgE...


In [10]:
photos_df.describe(include="all")


,business_alias,photo_url
count,1000,1000
unique,1000,1000
top,le-comptoir-de-la-gastronomie-paris,https://s3-media2.fl.yelpcdn.com/bphoto/Je6THJ...
freq,1,1


In [11]:
download_photos(photos_df, target_path="../data/raw/photos/")


Yelp API request failed with status code: 403.
Response text: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>AccessDenied</Code><Message>Access Denied</Message><RequestId>WEQ0P50D8WZD8KKG</RequestId><HostId>/z5pHo/Oh/o65XjXVUNVNLsKdr0eZ44HAFdfhZegnNZNxL+RW3teBfrZPVQabcXWmhjbXzUaK3o=</HostId></Error>


In [12]:
def one_hot_encode_list_variables(
    df: pd.DataFrame,
    columns: list[str],
) -> pd.DataFrame:
    """One-hot encode list variables.
    See : https://cmpoi.medium.com/a-quick-tutorial-to-encode-list-variables-125ba4040325

    - for each list variable
        - decode JSON values to list
        - make a dataframe of one-hot encoded values
        - append to original dataframe

    Args:
        df (pd.DataFrame): dataframe to encode
        columns (list[str]): list of columns to encode

    Raises:
        Exception: columns values should be (JSON encoded) lists of strings

    Returns:
        pd.DataFrame:  dataframe with encoded columns
    """
    df = df.copy()
    for col in columns:
        if not isinstance(df[col][0], list):
            df[col] = df[col].replace(
                "[]", "null"
            )  # replace empty list with null
            df[col] = df[col].apply(json.loads)  # convert string to list

        if not isinstance(df[col][0], list):
            raise Exception(f"{col} is not a list")

        categories_df = (
            pd.get_dummies(
                pd.DataFrame(
                    [
                        x
                        if x is not None
                        else ["__EMPTY__"]  # replace None with empty list
                        for x in df[col].tolist()
                    ]
                ).stack(),
            )
            .groupby(level=0)
            .sum()
        ).drop(
            columns="__EMPTY__", errors="ignore"
        )  # remove empty list

        df = pd.concat([df, categories_df], axis=1)

    return df


df = one_hot_encode_list_variables(
    businesses_df, ["business_categories", "business_parent_categories"]
)


In [13]:
df.describe(include="all")


,business_alias,business_review_count,business_rating,business_price,business_city,business_state,business_postal_code,business_country,business_latitude,business_longitude,...,jpsweets,latin,media,mediterranean,mexican,mideastern,nightlife,publicservicesgovt,restaurants,shopping
count,1000,1000.000000,1000.000000,1000.000000,1000,1000,1000,1000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
unique,1000,NaN,NaN,NaN,40,6,301,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,le-comptoir-de-la-gastronomie-paris,NaN,NaN,NaN,Paris,13,2000,AU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,200,200,139,200,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,529.395000,4.270500,2.144000,NaN,NaN,NaN,NaN,13.688472,35.219700,...,0.001000,0.001000,0.003000,0.003000,0.004000,0.003000,0.060000,0.001000,0.87900,0.002000
std,NaN,1134.869631,0.353135,0.897814,NaN,NaN,NaN,NaN,34.823799,93.358452,...,0.031623,0.031623,0.054717,0.054717,0.063151,0.054717,0.237606,0.031623,0.32629,0.044699
min,NaN,6.000000,3.000000,0.000000,NaN,NaN,NaN,NaN,-33.897026,-74.016022,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,NaN,30.000000,4.000000,2.000000,NaN,NaN,NaN,NaN,-22.983292,-43.218462,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000
50%,NaN,71.000000,4.500000,2.000000,NaN,NaN,NaN,NaN,35.673141,2.340317,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000
75%,NaN,265.750000,4.500000,3.000000,NaN,NaN,NaN,NaN,40.751259,139.770420,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000


## Commentaires clients


In [14]:
fig = px.histogram(reviews_df, x="review_rating", marginal="box")
fig.show()


In [15]:
from sklearn.model_selection import train_test_split


X = reviews_df["review_text"]
y = reviews_df["review_rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


### Dummy regression

In [27]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


from sklearn.preprocessing import StandardScaler


scaler = StandardScaler().fit(X_train_vec.toarray())

X_train_std = scaler.transform(X_train_vec.toarray())
X_test_std = scaler.transform(X_test_vec.toarray())


words_count = pd.Series(
    X_train_vec.sum(axis=0).tolist()[0],
    index=vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import ElasticNetCV


reg = ElasticNetCV(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
coefs = pd.Series(reg.coef_, index=vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()


y_pred = reg.predict(X_test_std)


from sklearn.metrics import median_absolute_error, r2_score


fig = px.box(
    x=y_test,
    y=y_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"Actual vs Predicted / R² = {round(r2_score(y_test, y_pred), 3)} / MAE = {round(median_absolute_error(y_test, y_pred), 3)}",
    color=y_test,
)
fig.show()


### More complex    

In [28]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer(
    strip_accents="unicode",
    stop_words="english",
    ngram_range=(1, 3),
    max_df=0.9,
    min_df=2,
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


from sklearn.preprocessing import StandardScaler


scaler = StandardScaler().fit(X_train_vec.toarray())

X_train_std = scaler.transform(X_train_vec.toarray())
X_test_std = scaler.transform(X_test_vec.toarray())


words_count = pd.Series(
    X_train_vec.sum(axis=0).tolist()[0],
    index=vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import ElasticNetCV


reg = ElasticNetCV(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
coefs = pd.Series(reg.coef_, index=vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()


y_pred = reg.predict(X_test_std)


from sklearn.metrics import median_absolute_error, r2_score


fig = px.box(
    x=y_test,
    y=y_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"Actual vs Predicted / R² = {round(r2_score(y_test, y_pred), 3)} / MAE = {round(median_absolute_error(y_test, y_pred), 3)}",
    color=y_test,
)
fig.show()


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords


nltk.download("stopwords")


vectorizer = CountVectorizer(
    strip_accents="unicode",
    stop_words=stopwords.words("english"),
    ngram_range=(1, 3),
    max_df=0.9,
    min_df=2,
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


from sklearn.preprocessing import StandardScaler


scaler = StandardScaler().fit(X_train_vec.toarray())

X_train_std = scaler.transform(X_train_vec.toarray())
X_test_std = scaler.transform(X_test_vec.toarray())


words_count = pd.Series(
    X_train_vec.sum(axis=0).tolist()[0],
    index=vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import ElasticNetCV


reg = ElasticNetCV(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
coefs = pd.Series(reg.coef_, index=vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()


y_pred = reg.predict(X_test_std)


from sklearn.metrics import median_absolute_error, r2_score


fig = px.box(
    x=y_test,
    y=y_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"Actual vs Predicted / R² = {round(r2_score(y_test, y_pred), 3)} / MAE = {round(median_absolute_error(y_test, y_pred), 3)}",
    color=y_test,
)
fig.show()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/clement/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

vectorizer = CountVectorizer(
    preprocessor=lambda x: PorterStemmer().stem(x)
    if x not in string.punctuation
    else None,
    stop_words=stopwords.words("english"),
    ngram_range=(1, 3),
    max_df=0.9,
    min_df=2,
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


from sklearn.preprocessing import StandardScaler


scaler = StandardScaler().fit(X_train_vec.toarray())

X_train_std = scaler.transform(X_train_vec.toarray())
X_test_std = scaler.transform(X_test_vec.toarray())


words_count = pd.Series(
    X_train_vec.sum(axis=0).tolist()[0],
    index=vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import ElasticNetCV


reg = ElasticNetCV(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
coefs = pd.Series(reg.coef_, index=vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()


y_pred = reg.predict(X_test_std)


from sklearn.metrics import median_absolute_error, r2_score


fig = px.box(
    x=y_test,
    y=y_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"Actual vs Predicted / R² = {round(r2_score(y_test, y_pred), 3)} / MAE = {round(median_absolute_error(y_test, y_pred), 3)}",
    color=y_test,
)
fig.show()


/home/clement/Workspace/oc_p6/env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.



In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string


vectorizer = CountVectorizer(
    preprocessor=lambda x: WordNetLemmatizer().lemmatize(x.lower())
    if x not in string.punctuation
    else None,
    stop_words=stopwords.words("english"),
    ngram_range=(1, 3),
    max_df=0.9,
    min_df=2,
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


from sklearn.preprocessing import StandardScaler


scaler = StandardScaler().fit(X_train_vec.toarray())

X_train_std = scaler.transform(X_train_vec.toarray())
X_test_std = scaler.transform(X_test_vec.toarray())


words_count = pd.Series(
    X_train_vec.sum(axis=0).tolist()[0],
    index=vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import ElasticNetCV


reg = ElasticNetCV(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
coefs = pd.Series(reg.coef_, index=vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()


y_pred = reg.predict(X_test_std)


from sklearn.metrics import median_absolute_error, r2_score


fig = px.box(
    x=y_test,
    y=y_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"Actual vs Predicted / R² = {round(r2_score(y_test, y_pred), 3)} / MAE = {round(median_absolute_error(y_test, y_pred), 3)}",
    color=y_test,
)
fig.show()


/home/clement/Workspace/oc_p6/env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.



In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import spacy


nlp = spacy.load("en_core_web_sm")


vectorizer = CountVectorizer(
    preprocessor=lambda x: " ".join(
        [
            token.lemma_
            for token in nlp(x.lower())
            if x not in string.punctuation 
        ]
    ),
    stop_words=stopwords.words("english"),
    ngram_range=(1, 3),
    max_df=0.9,
    min_df=2,
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


from sklearn.preprocessing import StandardScaler


scaler = StandardScaler().fit(X_train_vec.toarray())

X_train_std = scaler.transform(X_train_vec.toarray())
X_test_std = scaler.transform(X_test_vec.toarray())


words_count = pd.Series(
    X_train_vec.sum(axis=0).tolist()[0],
    index=vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import ElasticNetCV


reg = ElasticNetCV(random_state=42, n_jobs=-1).fit(X_train_std, y_train)
coefs = pd.Series(reg.coef_, index=vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()


y_pred = reg.predict(X_test_std)


from sklearn.metrics import median_absolute_error, r2_score


fig = px.box(
    x=y_test,
    y=y_pred,
    labels={"x": "Actual", "y": "Predicted"},
    title=f"Actual vs Predicted / R² = {round(r2_score(y_test, y_pred), 3)} / MAE = {round(median_absolute_error(y_test, y_pred), 3)}",
    color=y_test,
)
fig.show()


/home/clement/Workspace/oc_p6/env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['could', 'far', 'might', 'must', 'need', 'shall', 'win', 'would'] not in stop_words.



### Ultimate 


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS


nltk.download("stopwords")
nlp = spacy.load("en")

stopwords = list(STOP_WORDS)



count_vectorizer = CountVectorizer(
    strip_accents="unicode",
    lowercase=True,
    preprocessor=,
    tokenizer=,
    stop_words=,
    ngram_range=(1, 3),
)
X_count = count_vectorizer.fit_transform(corpus)

words_count = pd.Series(
    X_count.sum(axis=0).tolist()[0],
    index=count_vectorizer.get_feature_names(),
)

top_20_count = words_count.sort_values(ascending=False).head(20)

fig = px.bar(
    top_20_count,
    x=top_20_count.index,
    y=top_20_count.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_count.values,
)
fig.show()


from sklearn.linear_model import LinearRegression


reg = LinearRegression().fit(X_count, y)
coefs = pd.Series(reg.coef_, index=count_vectorizer.get_feature_names())

top_20_coefs = coefs.nlargest(10).append(coefs.nsmallest(10)).sort_values()

fig = px.bar(
    top_20_coefs,
    x=top_20_coefs.index,
    y=top_20_coefs.values,
    labels={"x": "Word", "y": "Count"},
    title="Top 20 words in reviews",
    color=top_20_coefs.values,
)
fig.show()
